In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import openai
import os, pprint
from functools import lru_cache
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
MONGODB_URI = os.getenv('MONGODB_URI')
db_name = os.getenv('MONGODB_DATABASE')
collection_name = os.getenv('MONGODB_TEMPUSER')
vector_search_idx = os.getenv('MONGODB_VECTOR_INDEX_TEMPUSER')

In [27]:
client = MongoClient(MONGODB_URI)
db = client[db_name]
collection = db[collection_name]

def get_user_embeddings(session_id):
    user_doc = collection.find_one({"session_id": session_id})
    if user_doc and "embeddings" in user_doc:
        return user_doc["embeddings"]
    return None

# Assuming 'session_id' is known and valid
session_embeddings = get_user_embeddings("_HrwNQCNinc_Ki6kMT3vkw")

In [28]:
vector_search = MongoDBAtlasVectorSearch(
   embedding=OpenAIEmbeddings(disallowed_special=()),
   collection=collection,  # Ensure this collection is correctly set up for vector search
   index_name=vector_search_idx,
   embedding_key = "embeddings"
)

retriever = vector_search.as_retriever(
   search_type="similarity",
   search_kwargs={"k": 5, "score_threshold": 0.75}
)


In [30]:
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer or if it is not provided in the context, just say that you don't know, don't try to make up an answer.
If the answer is in the context, don't say mentioned in the context.
Please provide a detailed explanation and if applicable, give examples or historical context.
{context}
Question: {question}
"""

custom_rag_prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI()

def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | custom_rag_prompt
   | llm
   | StrOutputParser()
)

In [31]:
MAX_CACHE_SIZE = 100
@lru_cache(maxsize=MAX_CACHE_SIZE)
def cached_query(question):
    response = rag_chain.invoke(question)
    return response

RED = '\033[91m'
GREEN = '\033[92m'
YELLOW = '\033[93m'
RESET = '\033[0m'


question = "What's my name?"
answer = cached_query(question) 

print(f"{YELLOW}Cache Info: {cached_query.cache_info()}{RESET}")
print(f"{RED}Question: {question}{RESET}")
print(f"{GREEN}Answer: {answer}{RESET}")

documents = retriever.get_relevant_documents(question)
print("\nSource documents:")
pprint.pprint(documents)

Cache Info: CacheInfo(hits=0, misses=1, maxsize=100, currsize=1)
Question: What's my name?
Answer: I'm sorry, but without any information provided about your name in the context, I don't know what your name is.

Source documents:
[]
